# Capstone Project - What food is around Hotels in Shanghai?
## Chosen by picking the hotels around the highest rated restaurant in Shanghai
## Then finding the restaurants around those results
### *Lawrence Sheed*

---

###Do our Setup Code

In [1]:
#Disable pip's annoying warnings
%env _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1 

#disable python warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#install libraries needed
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install geopy
!pip install geocoder

#import libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json
import folium 
from bs4 import BeautifulSoup  

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    

from sklearn.cluster import KMeans
from IPython.display import display_html
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import geocoder # import geocoder

print('Libraries imported.')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


env: _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1
Libraries imported.


### Login to FourSquare w/Credentials

In [2]:
#Foursquare login from separate file so we don't accidentally share keys
import config as cfg
CLIENT_ID = cfg.foursquare['client_id']
CLIENT_SECRET = cfg.foursquare['client_secret']
ACCESS_TOKEN = cfg.foursquare['access_token']

VERSION = '20180604'
LIMIT = 50
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK


## Pick somewhere central downtown in Shanghai

In [3]:
address = '1331 Fuxing Zhong Lu, Shanghai'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

#Completely innaccurate, sadly
print ('FourSquare is _wrong_ *CORRECT LAT/LNG RESULTS ARE* : 31.211398,121.453289')
#Actual Longtitude / Lat is
# 31.211398,121.453289 
latitude = 31.211398
longtitude = 121.453289 



31.2238233 121.4858025
FourSquare is _wrong_ *CORRECT LAT/LNG RESULTS ARE* : 31.211398,121.453289


## Search for restaurant venues in this area

##Search for restaurants in the area

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [4]:
search_query = 'Restaurant'
#5km's as FourSquare is rather empty for Shanghai
radius = 5000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK&client_secret=COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH&ll=31.211398,121.4858025&oauth_token=0JXEDHYX0LFM1M3P1XLVEV0TNGWOGPVCE4PKVR3E4LSX2MWA&v=20180604&query=Restaurant&radius=5000&limit=50'

In [6]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

display_html(dataframe.head())
print ("Row/Col Results: ",dataframe.shape)

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,location.neighborhood
0,57219542498eefea55c61543,陈记海鲜酒家 Chen Ji Seafood Restaurant,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1627413702,False,31.201891,121.482660,"[{'label': 'display', 'lat': 31.201891, 'lng':...",1099,CN,上海市,上海市,中国,"[上海市, 上海市]",NaN,NaN,NaN,NaN
1,4bce8945fb84c9b6f20f243e,Cantonese Cuisine Restaurant (家有靓场),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1627413702,False,31.209735,121.477438,"[{'label': 'display', 'lat': 31.209735, 'lng':...",817,CN,上海市,上海市,中国,"[丽园路501, 上海市, 上海市]",丽园路501,NaN,NaN,NaN
2,51ebd57b498e5a632e8bb90a,M'Z Restaurant & Café,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1627413702,False,31.203032,121.477878,"[{'label': 'display', 'lat': 31.20303195908394...",1198,CN,上海市,上海市,中国,"[局门路550号6102室, 上海市, 上海市]",局门路550号6102室,NaN,NaN,NaN
3,4f02f82830f899233531a663,DaMaTou Tea Restaurant (大马头茶餐厅),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1627413702,False,31.220979,121.478385,"[{'label': 'display', 'lat': 31.22097896730746...",1279,CN,Huangpu,上海市,中国,"[西藏南路577号 (近复兴东路), Huangpu, 上海市, 200021]",西藏南路577号,近复兴东路,200021,NaN
4,4e76de76c65bb91db4fd8b1c,Mari e Monti Italian Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1627413702,False,31.209967,121.472489,"[{'label': 'display', 'lat': 31.209967, 'lng':...",1277,CN,上海市,上海市,中国,"[Mengzi Road (Xujiahui Road), 上海市, 上海市]",Mengzi Road,Xujiahui Road,NaN,NaN


Row/Col Results:  (50, 18)


## Tidy up Data

In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
venues = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
venues['categories'] = venues.apply(get_category_type, axis=1)

# clean column names by keeping only last term
venues.columns = [column.split('.')[-1] for column in venues.columns]

print ("Show the Panda Data Store for 'venues' DataStore with Venue Names and location data")
display_html(venues.head(10))
print ("Row/Col Results: ",venues.shape)

Show the Panda Data Store for 'venues' DataStore with Venue Names and location data


,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,neighborhood,id
0,陈记海鲜酒家 Chen Ji Seafood Restaurant,Seafood Restaurant,31.201891,121.482660,"[{'label': 'display', 'lat': 31.201891, 'lng':...",1099,CN,上海市,上海市,中国,"[上海市, 上海市]",NaN,NaN,NaN,NaN,57219542498eefea55c61543
1,Cantonese Cuisine Restaurant (家有靓场),Chinese Restaurant,31.209735,121.477438,"[{'label': 'display', 'lat': 31.209735, 'lng':...",817,CN,上海市,上海市,中国,"[丽园路501, 上海市, 上海市]",丽园路501,NaN,NaN,NaN,4bce8945fb84c9b6f20f243e
2,M'Z Restaurant & Café,Bar,31.203032,121.477878,"[{'label': 'display', 'lat': 31.20303195908394...",1198,CN,上海市,上海市,中国,"[局门路550号6102室, 上海市, 上海市]",局门路550号6102室,NaN,NaN,NaN,51ebd57b498e5a632e8bb90a
3,DaMaTou Tea Restaurant (大马头茶餐厅),Chinese Restaurant,31.220979,121.478385,"[{'label': 'display', 'lat': 31.22097896730746...",1279,CN,Huangpu,上海市,中国,"[西藏南路577号 (近复兴东路), Huangpu, 上海市, 200021]",西藏南路577号,近复兴东路,200021,NaN,4f02f82830f899233531a663
4,Mari e Monti Italian Restaurant,Italian Restaurant,31.209967,121.472489,"[{'label': 'display', 'lat': 31.209967, 'lng':...",1277,CN,上海市,上海市,中国,"[Mengzi Road (Xujiahui Road), 上海市, 上海市]",Mengzi Road,Xujiahui Road,NaN,NaN,4e76de76c65bb91db4fd8b1c
5,新旺茶餐厅 Xinwang Restaurant,Chinese Restaurant,31.207974,121.465385,"[{'label': 'display', 'lat': 31.20797425388117...",1980,CN,上海市,上海市,中国,"[卢湾区徐家汇路550弄1号宝鼎大厦101室 (打浦店), 上海市, 上海市]",卢湾区徐家汇路550弄1号宝鼎大厦101室,打浦店,NaN,NaN,4b9c843ff964a520d06d36e3
6,Cha's Restaurant (查餐厅),Hong Kong Restaurant,31.220636,121.461609,"[{'label': 'display', 'lat': 31.22063626268351...",2522,CN,Huangpu,上海市,中国,"[30 Sinan Rd, 1F | 思南路30-4号 (Huahai Rd | 淮海中路)...","30 Sinan Rd, 1F | 思南路30-4号",Huahai Rd | 淮海中路,200020,NaN,4b0588bef964a52050d722e3
7,Qian Restaurant (黔香阁),Guizhou Restaurant,31.226825,121.473713,"[{'label': 'display', 'lat': 31.22682469090645...",2067,CN,Huangpu,上海市,中国,[93-111 Huaihai M Rd | 淮海中路93-111号 (Times Squa...,93-111 Huaihai M Rd | 淮海中路93-111号,Times Square | 大上海时代广场,200021,Huáihǎi Zhōnglù,5b0a25db418686002c623488
8,Chao Man Restaurant (潮民冰室) (潮民冰室),Hong Kong Restaurant,31.226804,121.473690,"[{'label': 'display', 'lat': 31.226804, 'lng':...",2066,CN,Huangpu,上海市,中国,"[Huangpu, 上海市]",NaN,NaN,NaN,NaN,58be8f2a5490d37e4252b13f
9,Super Star Seafood Restaurant (鸿星荟海鲜酒家),Seafood Restaurant,31.225057,121.469323,"[{'label': 'display', 'lat': 31.22505721460541...",2184,CN,Huangpu,上海市,中国,[300 Huaihai M Rd | 淮海中路300号 (4/F-05 K11 Art M...,300 Huaihai M Rd | 淮海中路300号,4/F-05 K11 Art Mall | 上海K11购物艺术中心,200021,Huáihǎi Zhōnglù,5129a540e4b040b1d138f2e6


Row/Col Results:  (50, 16)


## Lets add the ratings to the Venues (note due to API call limits potentially being reached, results might *all* be unrated)


In [8]:

#create a new column called rating, and set the default value
venues['rating'] = 0

#loop through our venue's array, and search each one for ratings
for index in venues.index:
    venue_id = venues['id'][index]

    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

    rating = requests.get(url).json()
    try:
        print("Name: ", result['response']['venue']['name'])
        print("Rating: ",result['response']['venue']['rating'])
        #update the rating for this result
        venues['rating'][index]=result['response']['venue']['rating']

    except:
        print('Venue index',index,'is unrated')
        
    #print (rating)
    #print (index)




Venue index 0 is unrated
Venue index 1 is unrated
Venue index 2 is unrated
Venue index 3 is unrated
Venue index 4 is unrated
Venue index 5 is unrated
Venue index 6 is unrated
Venue index 7 is unrated
Venue index 8 is unrated
Venue index 9 is unrated
Venue index 10 is unrated
Venue index 11 is unrated
Venue index 12 is unrated
Venue index 13 is unrated
Venue index 14 is unrated
Venue index 15 is unrated
Venue index 16 is unrated
Venue index 17 is unrated
Venue index 18 is unrated
Venue index 19 is unrated
Venue index 20 is unrated
Venue index 21 is unrated
Venue index 22 is unrated
Venue index 23 is unrated
Venue index 24 is unrated
Venue index 25 is unrated
Venue index 26 is unrated
Venue index 27 is unrated
Venue index 28 is unrated
Venue index 29 is unrated
Venue index 30 is unrated
Venue index 31 is unrated
Venue index 32 is unrated
Venue index 33 is unrated
Venue index 34 is unrated
Venue index 35 is unrated
Venue index 36 is unrated
Venue index 37 is unrated
Venue index 38 is unra

## Show the names,id's of the venues, in descending rated order

In [9]:
newdf = venues[['name','id','rating']]
display_html(newdf.sort_values(by='rating', ascending=False))
highest_rating_id = newdf['id'][0]



,name,id,rating
0,陈记海鲜酒家 Chen Ji Seafood Restaurant,57219542498eefea55c61543,0
37,Sports Bar & Restaurant,5331742d498e9f8f33518f7d,0
27,Memory Restaurant (年代秀),5254dfe3498e1ef2516e55c6,0
28,Xin Ji Shi Restaurant (新吉士酒樓),4cc6c8891e596dcbf569e067,0
29,Charme Restaurant (港丽餐厅),51d7f9c8498e73d48c4deeb2,0
30,Jia Ming Restaurant (佳鸣酒楼),54ae0921498e554ac662bfc4,0
31,QIANQIU Restaurant (千秋膳房),4f65a9dee4b0c65cc56098d3,0
32,Long Restaurant (陇翠轩),5545a050498e2185e6fcf404,0
33,Dragon King Restaurant (龙皇),4ccace181ce5236ab767c165,0
34,Shanghai No.1 Rest. (上海1号私藏菜),4b74dbe3f964a5209df52de3,0


## Draw Restaurants on Map, with best one highlighted

In [10]:
print ("Highest Rated ID: ", highest_rating_id) #Sorted already by highest rating

#Get the array index
index = venues.loc[venues['id'] == highest_rating_id].index[0]
latitude = venues['lat'][index]
longitude = venues ['lng'][index]
name = venues['name'][index]
radius = 2000 #1km
print ("Best Venue Name: ", venues['name'][index])


Highest Rated ID:  57219542498eefea55c61543
Best Venue Name:  陈记海鲜酒家 Chen Ji Seafood Restaurant


In [11]:
display_html(venues.head(5))

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the  复兴中路

#Add best rest in red.
folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        color='blue',
        popup=name,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)
# add the FourSquare Data as blue circle markers
#unfortunately folium seems to break hard if chinese chars are in the label
for lat, lng, category in zip(venues.lat, venues.lng, venues.categories):
    label = ' {}'.format(category)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,neighborhood,id,rating
0,陈记海鲜酒家 Chen Ji Seafood Restaurant,Seafood Restaurant,31.201891,121.482660,"[{'label': 'display', 'lat': 31.201891, 'lng':...",1099,CN,上海市,上海市,中国,"[上海市, 上海市]",NaN,NaN,NaN,NaN,57219542498eefea55c61543,0
1,Cantonese Cuisine Restaurant (家有靓场),Chinese Restaurant,31.209735,121.477438,"[{'label': 'display', 'lat': 31.209735, 'lng':...",817,CN,上海市,上海市,中国,"[丽园路501, 上海市, 上海市]",丽园路501,NaN,NaN,NaN,4bce8945fb84c9b6f20f243e,0
2,M'Z Restaurant & Café,Bar,31.203032,121.477878,"[{'label': 'display', 'lat': 31.20303195908394...",1198,CN,上海市,上海市,中国,"[局门路550号6102室, 上海市, 上海市]",局门路550号6102室,NaN,NaN,NaN,51ebd57b498e5a632e8bb90a,0
3,DaMaTou Tea Restaurant (大马头茶餐厅),Chinese Restaurant,31.220979,121.478385,"[{'label': 'display', 'lat': 31.22097896730746...",1279,CN,Huangpu,上海市,中国,"[西藏南路577号 (近复兴东路), Huangpu, 上海市, 200021]",西藏南路577号,近复兴东路,200021,NaN,4f02f82830f899233531a663,0
4,Mari e Monti Italian Restaurant,Italian Restaurant,31.209967,121.472489,"[{'label': 'display', 'lat': 31.209967, 'lng':...",1277,CN,上海市,上海市,中国,"[Mengzi Road (Xujiahui Road), 上海市, 上海市]",Mengzi Road,Xujiahui Road,NaN,NaN,4e76de76c65bb91db4fd8b1c,0


## Select hotels around the best Venue

In [12]:

#Get the array index
index = venues.loc[venues['id'] == highest_rating_id].index[0]
latitude = venues['lat'][index]
longitude = venues ['lng'][index]
name = venues['name'][index]

#Make our call
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&query="Hotel"'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK&client_secret=COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH&ll=31.201891,121.48266&v=20180604&radius=2000&limit=50&query="Hotel"'

## How many results?

In [13]:
import requests

results = requests.get(url).json()
'There are {} Hotel results around {}.'.format(len(results['response']['groups'][0]['items']),name)


'There are 19 Hotel results around 陈记海鲜酒家 Chen Ji Seafood Restaurant.'

In [14]:
items = results['response']['groups'][0]['items']


## Convert those to Dataframe and display the first 5

In [15]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]


display_html (dataframe_filtered.head(5))




,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Pullman Shanghai Skyway Hotel,Hotel,"打浦路15号, 浦东新区",Xujiahui Rd.,31.204089,121.471604,"[{'label': 'display', 'lat': 31.204089, 'lng':...",1080,200023,CN,上海市,上海市,中国,"[打浦路15号, 浦东新区 (Xujiahui Rd.), 上海市, 上海市, 200023...",NaN,4bc7f52092b376b0eec1503a
1,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),Hotel,1188 Xueye Rd,Shibo Avenue,31.200941,121.500556,"[{'label': 'display', 'lat': 31.20094111230248...",1707,NaN,CN,Huangpu,上海市,中国,"[1188 Xueye Rd (Shibo Avenue), Huangpu, 上海市, 中国]",NaN,50360802e4b098c76ebd50a7
2,Shanghai Marriott Riverside Hotel (上海绿地万豪酒店),Hotel,江滨路99号,NaN,31.193658,121.468057,"[{'label': 'display', 'lat': 31.19365803154623...",1665,200023,CN,上海市,上海市,中国,"[江滨路99号, 上海市, 上海市, 200023, 中国]",NaN,4de9c4e088774880e30b1eea
3,Ji Hotel (全季酒店),Hotel,"肇嘉浜路221号, 徐汇区",NaN,31.202784,121.463229,"[{'label': 'display', 'lat': 31.202784, 'lng':...",1852,200032,CN,上海市,上海市,中国,"[肇嘉浜路221号, 徐汇区, Lúwān, 上海市, 上海市, 200032, 中国]",Lúwān,5249242d11d29c723b7fc0bb
4,上海黄金岛大酒店,Hotel,制造局路,NaN,31.201406,121.481498,"[{'label': 'display', 'lat': 31.20140588577529...",123,NaN,CN,Huangpu,上海市,中国,"[制造局路, Huangpu, 上海市, 中国]",NaN,4e5644cbb0fbb294aaa5a0a1


## Draw Hotels on Map

In [16]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around restaurant


# add best Restaurant as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup=name,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [17]:

k=6
hotels = dataframe_filtered[dataframe_filtered['categories'].str.contains('Hotel',regex=False)]
#name	categories	address	lat	lng	labeledLatLngs	distance	postalCode	cc	city	state	country	formattedAddress	crossStreet	neighborhood	id
hotel_clustering = hotels.drop(['name','categories','address','labeledLatLngs','distance','postalCode','cc','city','state','country','formattedAddress','crossStreet','neighborhood','id'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(hotel_clustering)

print ("KMEANS array", kmeans.labels_)

hotels.insert(0, 'KMeansCluster', kmeans.labels_)

hotels

KMEANS array [5 3 0 0 5 5 3 2 0 0 4 1 2 2 1 4 0]


,KMeansCluster,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,5,Pullman Shanghai Skyway Hotel,Hotel,"打浦路15号, 浦东新区",Xujiahui Rd.,31.204089,121.471604,"[{'label': 'display', 'lat': 31.204089, 'lng':...",1080,200023,CN,上海市,上海市,中国,"[打浦路15号, 浦东新区 (Xujiahui Rd.), 上海市, 上海市, 200023...",NaN,4bc7f52092b376b0eec1503a
1,3,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),Hotel,1188 Xueye Rd,Shibo Avenue,31.200941,121.500556,"[{'label': 'display', 'lat': 31.20094111230248...",1707,NaN,CN,Huangpu,上海市,中国,"[1188 Xueye Rd (Shibo Avenue), Huangpu, 上海市, 中国]",NaN,50360802e4b098c76ebd50a7
2,0,Shanghai Marriott Riverside Hotel (上海绿地万豪酒店),Hotel,江滨路99号,NaN,31.193658,121.468057,"[{'label': 'display', 'lat': 31.19365803154623...",1665,200023,CN,上海市,上海市,中国,"[江滨路99号, 上海市, 上海市, 200023, 中国]",NaN,4de9c4e088774880e30b1eea
3,0,Ji Hotel (全季酒店),Hotel,"肇嘉浜路221号, 徐汇区",NaN,31.202784,121.463229,"[{'label': 'display', 'lat': 31.202784, 'lng':...",1852,200032,CN,上海市,上海市,中国,"[肇嘉浜路221号, 徐汇区, Lúwān, 上海市, 上海市, 200032, 中国]",Lúwān,5249242d11d29c723b7fc0bb
4,5,上海黄金岛大酒店,Hotel,制造局路,NaN,31.201406,121.481498,"[{'label': 'display', 'lat': 31.20140588577529...",123,NaN,CN,Huangpu,上海市,中国,"[制造局路, Huangpu, 上海市, 中国]",NaN,4e5644cbb0fbb294aaa5a0a1
5,5,Magnotel,Hotel,斜土路 | Xietu Rd.,Mangzi Lu,31.204249,121.474147,"[{'label': 'display', 'lat': 31.20424914351182...",851,NaN,CN,Huangpu,上海市,中国,"[斜土路 | Xietu Rd. (Mangzi Lu), Huangpu, 上海市, 中国]",NaN,4cac7781a6e08cfa165aab94
6,3,URSIDE Hotel & Café,Hotel,"Unit 3A, Block 4, 60 Huayuangang Road | 花园港路60...",Ban songyuan Road | 半淞虹园路,31.204022,121.492352,"[{'label': 'display', 'lat': 31.20402151063051...",952,NaN,CN,Huangpu,上海市,中国,"[Unit 3A, Block 4, 60 Huayuangang Road | 花园港路6...",NaN,5bd43706e55d8b002ce1eb2a
7,2,全季酒店 Ji Hotel,Hotel,"重庆南路283号, 黄浦区",徐家汇路,31.209369,121.473307,"[{'label': 'display', 'lat': 31.209369, 'lng':...",1219,200025,CN,上海市,上海市,中国,"[重庆南路283号, 黄浦区 (徐家汇路), 上海市, 上海市, 200025, 中国]",NaN,4d0ce162bdbfa35d7e436672
8,0,Atour S Hotel (Atour S Hotel 亚朵酒店),Hotel,斜土路688号,上海市卢湾区斜土路688号,31.204238,121.468373,"[{'label': 'display', 'lat': 31.20423812686419...",1385,NaN,CN,Huangpu,上海市,中国,"[斜土路688号 (上海市卢湾区斜土路688号), Huangpu, 上海市, 中国]",NaN,4dbd303443a1d8504b9f67a0
9,0,Ramada Peace Hotel Shanghai,Hotel,679号 Xujiahui Rd,NaN,31.204445,121.468261,"[{'label': 'display', 'lat': 31.20444534386017...",1400,NaN,CN,Huangpu,上海市,中国,"[679号 Xujiahui Rd, Huangpu, 上海市, 中国]",NaN,4bf08355b315c9b6e61d93ff


## Draw a KMeans Cluster map of Hotel Distribution around the best restaurant

In [18]:
# create map
map_clusters = folium.Map(location=[latitude,longitude],zoom_start=14)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []


#Loop through our KMEANS Cluster
for lat, lng, myName, cluster, address in zip(hotels['lat'], hotels['lng'], hotels['name'], hotels['KMeansCluster'], hotels['address']):
    label = ' Cluster ' + str(cluster)+ ' {}, {}'.format(myName, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

#Get the array index
index = venues.loc[venues['id'] == highest_rating_id].index[0]
latitude = venues['lat'][index]
longitude = venues ['lng'][index]
name = venues['name'][index]

#Add our original marker
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup=name,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(map_clusters)

map_clusters


## Use FourSquare API to check whats around each hotel

In [19]:
radius = 500
LIMIT = 200

#Reset Venues again
venues = []
for lat, lng, myName in zip(hotels['lat'], hotels['lng'], hotels['name']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            myName,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


## How many results around Hotels clusters now?

In [20]:
#Excerpt our results
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Hotel', 'HotelLat', 'HotelLong','VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
display_html(venues_df.head())
print (venues_df.shape)


,Hotel,HotelLat,HotelLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Xing Guang Photographic Equipment City (星光摄影器材城),31.205013,121.470165,Camera Store
1,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Hotel
2,Pullman Shanghai Skyway Hotel,31.204089,121.471604,聪哥小馆,31.204182,121.466744,Asian Restaurant
3,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Blue Mountain Youth Hostel,31.201072,121.470156,Hostel
4,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Century Mart (世纪联华),31.204604,121.469333,Big Box Store


(298, 7)


## Filter result set to only contain restaurants near our hotels

In [21]:
df1 = venues_df.loc[venues_df['VenueCategory'].str.contains('Restaurant') == True]
display_html(df1.head())
print (df1.shape)



,Hotel,HotelLat,HotelLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory
2,Pullman Shanghai Skyway Hotel,31.204089,121.471604,聪哥小馆,31.204182,121.466744,Asian Restaurant
7,Pullman Shanghai Skyway Hotel,31.204089,121.471604,McDonald's (麦当劳),31.201280,121.472769,Fast Food Restaurant
8,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Remedy 365 (源氣 365),31.207406,121.473457,Restaurant
9,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Mr. Pancake,31.207638,121.473523,American Restaurant
11,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Top Chef,31.207890,121.473130,Italian Restaurant


(104, 7)


## Sort by Category

In [22]:
df1.sort_values(by=['VenueCategory'],inplace=True)
df1.head()

,Hotel,HotelLat,HotelLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory
91,全季酒店 Ji Hotel,31.209369,121.473307,Mr. Pancake,31.207638,121.473523,American Restaurant
9,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Mr. Pancake,31.207638,121.473523,American Restaurant
64,Magnotel,31.204249,121.474147,Mr. Pancake,31.207638,121.473523,American Restaurant
21,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),31.200941,121.500556,多利萝玛 Tony Roma's,31.198010,121.500630,American Restaurant
105,Atour S Hotel (Atour S Hotel 亚朵酒店),31.204238,121.468373,聪哥小馆,31.204182,121.466744,Asian Restaurant


## Make a onehot on the restaurant type

In [23]:
# Creating Dummy Variables for Venue Category
onehot = pd.get_dummies(df1['VenueCategory'],prefix='',prefix_sep='')
new = df1
onehot.head(15)

,American Restaurant,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Hainan Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Moroccan Restaurant,Restaurant,Shanghai Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Thai Restaurant,Yunnan Restaurant
91,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
105,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
123,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
276,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
new.head()

,Hotel,HotelLat,HotelLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory
91,全季酒店 Ji Hotel,31.209369,121.473307,Mr. Pancake,31.207638,121.473523,American Restaurant
9,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Mr. Pancake,31.207638,121.473523,American Restaurant
64,Magnotel,31.204249,121.474147,Mr. Pancake,31.207638,121.473523,American Restaurant
21,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),31.200941,121.500556,多利萝玛 Tony Roma's,31.198010,121.500630,American Restaurant
105,Atour S Hotel (Atour S Hotel 亚朵酒店),31.204238,121.468373,聪哥小馆,31.204182,121.466744,Asian Restaurant


## Merge the onehot to a new PD

In [25]:
merged = pd.merge(new,onehot,on=new.index)

In [26]:
display_html(merged.head(10))
print(merged.shape)

,key_0,Hotel,HotelLat,HotelLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory,American Restaurant,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Hainan Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Moroccan Restaurant,Restaurant,Shanghai Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Thai Restaurant,Yunnan Restaurant
0,91,全季酒店 Ji Hotel,31.209369,121.473307,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,64,Magnotel,31.204249,121.474147,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,21,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),31.200941,121.500556,多利萝玛 Tony Roma's,31.198010,121.500630,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,105,Atour S Hotel (Atour S Hotel 亚朵酒店),31.204238,121.468373,聪哥小馆,31.204182,121.466744,Asian Restaurant,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,123,Ramada Peace Hotel Shanghai,31.204445,121.468261,聪哥小馆,31.204182,121.466744,Asian Restaurant,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,50,上海黄金岛大酒店,31.201406,121.481498,Cai Xian Sheng (蔡先生传统小吃),31.204786,121.480606,Asian Restaurant,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,34,Ji Hotel (全季酒店),31.202784,121.463229,聪哥小馆,31.204182,121.466744,Asian Restaurant,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2,Pullman Shanghai Skyway Hotel,31.204089,121.471604,聪哥小馆,31.204182,121.466744,Asian Restaurant,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,276,鼎园瑞峰公寓酒店,31.203927,121.462582,聪哥小馆,31.204182,121.466744,Asian Restaurant,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(104, 31)


## Remove unnecessary key 0 column

In [27]:
merged.drop(['key_0'],axis=1,inplace=True)

In [28]:
merged.head(5)

,Hotel,HotelLat,HotelLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory,American Restaurant,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Hainan Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Moroccan Restaurant,Restaurant,Shanghai Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Thai Restaurant,Yunnan Restaurant
0,全季酒店 Ji Hotel,31.209369,121.473307,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Magnotel,31.204249,121.474147,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),31.200941,121.500556,多利萝玛 Tony Roma's,31.198010,121.500630,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Atour S Hotel (Atour S Hotel 亚朵酒店),31.204238,121.468373,聪哥小馆,31.204182,121.466744,Asian Restaurant,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Merge in the KMEANS data from the Restaurant types near our Hotels


In [29]:
clustering = merged.drop(['Hotel','HotelLat','HotelLong','VenueName','VenueLatitude','VenueLongitude','VenueCategory'],axis=1)


In [30]:
k=5
kmeans = KMeans(n_clusters=k,random_state=0).fit(clustering)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [31]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 4, 4, 4, 4, 4, 4, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [32]:
merged.insert(0,'ClusterLabels', kmeans.labels_)

## Draw Cluster Map of the Hotels and Restaurants (Hotels have a Blue surrounding circle)

In [33]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, name, cat, cluster in zip(merged['VenueLatitude'], merged['VenueLongitude'], merged['VenueName'],merged['VenueCategory'], merged['ClusterLabels']):
    label = folium.Popup(name +','+ cat + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
for lat, lng, hotel  in zip(merged['HotelLat'], merged['HotelLong'], merged['Hotel']):
    label = '{}'.format(hotel)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters) 


map_clusters

## Lets look at the data set results

In [34]:
clustering = merged.loc[merged['ClusterLabels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

In [35]:
clustering['VenueCategory'].value_counts()

Restaurant              6
Asian Restaurant        6
Hotpot Restaurant       6
Cantonese Restaurant    5
Szechuan Restaurant     5
American Restaurant     4
Indian Restaurant       3
Thai Restaurant         3
Japanese Restaurant     3
French Restaurant       3
Dim Sum Restaurant      3
Hainan Restaurant       2
Moroccan Restaurant     2
German Restaurant       2
Shanghai Restaurant     2
Taiwanese Restaurant    2
Korean Restaurant       1
Sushi Restaurant        1
Yunnan Restaurant       1
Name: VenueCategory, dtype: int64

In [36]:
clustering_1 = merged.loc[merged['ClusterLabels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

print(clustering_1['VenueCategory'].value_counts())
display_html (clustering_1)

Chinese Restaurant    19
Name: VenueCategory, dtype: int64


,Hotel,VenueLatitude,VenueLongitude,VenueCategory,American Restaurant,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Hainan Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Moroccan Restaurant,Restaurant,Shanghai Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Thai Restaurant,Yunnan Restaurant
15,鼎园瑞峰公寓酒店,31.202984,121.462459,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,鼎园瑞峰公寓酒店,31.208004,121.463109,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,Ramada Peace Hotel Shanghai,31.207974,121.465385,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,汉庭快捷酒店 HanTing Express,31.215781,121.478057,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,汉庭海友客栈(新天地店),31.209803,121.465410,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,Shanghai Celadon Theme Hotel (上海青瓷主题酒店),31.220979,121.478385,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21,鼎园瑞峰公寓酒店,31.208259,121.463950,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,汉庭海友客栈(新天地店),31.207974,121.465385,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,汉庭海友客栈(新天地店),31.209385,121.472665,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,全季酒店 Ji Hotel,31.209385,121.472665,Chinese Restaurant,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
clustering_2 = merged.loc[merged['ClusterLabels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

print (clustering_2['VenueCategory'].value_counts())
display_html (clustering_2)


Fast Food Restaurant    10
Name: VenueCategory, dtype: int64


,Hotel,VenueLatitude,VenueLongitude,VenueCategory,American Restaurant,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Hainan Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Moroccan Restaurant,Restaurant,Shanghai Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Thai Restaurant,Yunnan Restaurant
43,Hanting Seasons Hotel,31.213563,121.472696,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44,鼎园瑞峰公寓酒店,31.207431,121.464002,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
45,Ji Hotel (全季酒店),31.207145,121.463253,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46,汉庭海友客栈(新天地店),31.213563,121.472696,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
47,Pullman Shanghai Skyway Hotel,31.201280,121.472769,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48,汉庭快捷酒店 HanTing Express,31.213443,121.479089,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49,鼎园瑞峰公寓酒店,31.207145,121.463253,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50,汉庭快捷酒店 HanTing Express,31.212560,121.482006,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51,Magnotel,31.201280,121.472769,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52,全季酒店 Ji Hotel,31.213563,121.472696,Fast Food Restaurant,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
clustering_3 = merged.loc[merged['ClusterLabels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

clustering_3['VenueCategory'].value_counts()

Italian Restaurant    9
Name: VenueCategory, dtype: int64

In [39]:
clustering_4 = merged.loc[merged['ClusterLabels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

clustering_4['VenueCategory'].value_counts()

Dumpling Restaurant    6
Name: VenueCategory, dtype: int64

In [40]:
print('The merged set has {} unique hotels and {} restaurants.'.format(
        len(merged['Hotel'].unique()),
        merged.shape[0]
    ))

display_html (merged.head())
print (merged.shape)


The merged set has 15 unique hotels and 104 restaurants.


,ClusterLabels,Hotel,HotelLat,HotelLong,VenueName,VenueLatitude,VenueLongitude,VenueCategory,American Restaurant,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Hainan Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Moroccan Restaurant,Restaurant,Shanghai Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Thai Restaurant,Yunnan Restaurant
0,0,全季酒店 Ji Hotel,31.209369,121.473307,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,Pullman Shanghai Skyway Hotel,31.204089,121.471604,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,Magnotel,31.204249,121.474147,Mr. Pancake,31.207638,121.473523,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),31.200941,121.500556,多利萝玛 Tony Roma's,31.198010,121.500630,American Restaurant,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,Atour S Hotel (Atour S Hotel 亚朵酒店),31.204238,121.468373,聪哥小馆,31.204182,121.466744,Asian Restaurant,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(104, 31)


In [41]:

hrc = merged[['Hotel','VenueCategory']].groupby(['Hotel']).agg(['count']).sort_index(axis=0, ascending=True)

print ("Hotels with the most Restaurants around them")
#display_html (hrc)

print(list(hrc.columns.values.tolist()))
hrc.sort_index(axis=1, ascending=False)

Hotels with the most Restaurants around them
[('VenueCategory', 'count')]


,VenueCategory
,count
Hotel,
Atour S Hotel (Atour S Hotel 亚朵酒店),4
Hanting Seasons Hotel,16
Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),4
Ji Hotel (全季酒店),4
Magnotel,7
Pullman Shanghai Skyway Hotel,5
Ramada Peace Hotel Shanghai,6
Shanghai Celadon Theme Hotel (上海青瓷主题酒店),5


In [42]:
restaurants_hotel_count = merged[['Hotel','VenueCategory']].groupby(['VenueCategory']).agg(['count'])
print ("Restaurants with the most Hotels around them")
display_html (restaurants_hotel_count)


Restaurants with the most Hotels around them


,Hotel
,count
VenueCategory,
American Restaurant,4
Asian Restaurant,6
Cantonese Restaurant,5
Chinese Restaurant,19
Dim Sum Restaurant,3
Dumpling Restaurant,6
Fast Food Restaurant,10
French Restaurant,3


## We can make the following observations from the data

There are on average 7 restaurants around each hotel (15 unique hotels and 104 restaurants - 104/15 = 7)

The hotel with the highest number of restaurants is the Hanting Seasons Hotel, with 16 Restaurants Closeby.

The most popular restaurants near hotels are Chinese Restaurants, with 19 Chinese restaurants near Hotels.

1.  The most popular restaurants around the hotels are. Chinese Restaurants (who could have guessed!) (19 results)
2.  Then Fast Food (10 results)
3.  Then Italian Restaurants (9 results)
4.  Then Dumpling Restaurants (6 results)

Then in order

```Restaurant              6
Asian Restaurant        6
Hotpot Restaurant       6
Cantonese Restaurant    5
Szechuan Restaurant     5
American Restaurant     4
Indian Restaurant       3
Thai Restaurant         3
Japanese Restaurant     3
French Restaurant       3
Dim Sum Restaurant      3
Hainan Restaurant       2
Moroccan Restaurant     2
German Restaurant       2
Shanghai Restaurant     2
Taiwanese Restaurant    2
Korean Restaurant       1
Sushi Restaurant        1
Yunnan Restaurant       1
```

5. It might make sense to open up a Sushi, Korean, or Yunnan Restaurant near a hotel.
